In [1]:
import pandas as pd
from lib.preprocessing import *
from lib.cleaning import *
import seaborn as sns


# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer
# from mpl_toolkits.basemap import Basemap
# import matplotlib.pyplot as plt
# %matplotlib inline
# import missingno as msno

In [2]:
df = load_sample(percentage=1, csv_file="data/train_full.csv")
df

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),...,Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_21208,User_Id_1792,Bike,3,Business,9,3,10:32:58 AM,9,3,...,11:48:12 AM,19,25.3,NaN,-1.309445,36.910771,-1.280912,36.785252,Rider_Id_237,2994
1,Order_No_18662,User_Id_3631,Bike,3,Personal,10,5,12:17:47 PM,10,5,...,12:34:42 PM,2,NaN,NaN,-1.299082,36.765487,-1.296310,36.768822,Rider_Id_327,455
2,Order_No_24475,User_Id_627,Bike,3,Business,16,2,11:40:23 AM,16,2,...,12:11:43 PM,2,23.6,NaN,-1.255189,36.782203,-1.258099,36.794152,Rider_Id_56,2
3,Order_No_6605,User_Id_2532,Bike,3,Business,10,5,3:59:24 PM,10,5,...,5:15:27 PM,3,NaN,NaN,-1.257147,36.795063,-1.275311,36.813018,Rider_Id_547,2187
4,Order_No_4939,User_Id_633,Bike,3,Business,19,2,12:01:29 PM,19,2,...,12:35:57 PM,5,27.7,NaN,-1.300406,36.829741,-1.322800,36.830644,Rider_Id_111,492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21196,Order_No_10373,User_Id_1471,Bike,3,Business,13,3,1:28:29 PM,13,3,...,2:01:19 PM,4,27.0,NaN,-1.255189,36.782203,-1.261863,36.810023,Rider_Id_279,1177
21197,Order_No_2429,User_Id_3578,Bike,3,Business,27,6,12:53:55 PM,27,6,...,1:23:21 PM,1,NaN,NaN,-1.264960,36.798178,-1.252796,36.800313,Rider_Id_641,464
21198,Order_No_6918,User_Id_754,Bike,3,Business,10,3,1:01:48 PM,10,3,...,2:38:18 PM,5,30.4,NaN,-1.317387,36.811204,-1.297739,36.803145,Rider_Id_438,465
21199,Order_No_6919,User_Id_1425,Bike,3,Business,26,5,4:51:55 PM,26,5,...,6:04:04 PM,10,NaN,NaN,-1.338283,36.887177,-1.300406,36.829741,Rider_Id_68,1364


In [3]:
df = combine_weekdays(impute_temperature(drops(df)))
df.head(3)

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but IterativeImputer was fitted without feature names
  warnings.warn(


,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Time,Arrival at Pickup - Time,...,Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival,Fulfillment - Weekday (Su = 0),Fulfillment - Day of Month
0,Order_No_21208,User_Id_1792,Bike,3,Business,9,3,632,10:33:46 AM,10:52:10 AM,...,19,25.300000,-1.309445,36.910771,-1.280912,36.785252,Rider_Id_237,2994,3,9
1,Order_No_18662,User_Id_3631,Bike,3,Personal,10,5,737,12:18:24 PM,12:25:32 PM,...,2,22.910955,-1.299082,36.765487,-1.296310,36.768822,Rider_Id_327,455,5,10
2,Order_No_24475,User_Id_627,Bike,3,Business,16,2,700,11:40:38 AM,11:54:57 AM,...,2,23.600000,-1.255189,36.782203,-1.258099,36.794152,Rider_Id_56,2,2,16


In [4]:
df.loc[(df['Placement - Day of Month'] < df["Fulfillment - Day of Month"])]

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Time,Arrival at Pickup - Time,...,Distance (KM),Temperature,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival,Fulfillment - Weekday (Su = 0),Fulfillment - Day of Month
12786,Order_No_2800,User_Id_1402,Bike,3,Business,17,1,874,1:16:53 PM,1:59:45 PM,...,11,25.6,-1.225322,36.808550,-1.281754,36.832423,Rider_Id_928,2640,2,18
20466,Order_No_3530,User_Id_2457,Bike,3,Business,11,2,742,1:27:30 PM,1:32:38 PM,...,8,22.5,-1.291548,36.829752,-1.255189,36.782203,Rider_Id_247,1854,4,13


In [5]:
save_data(df, sql_db="data/cleaned_nelson.db")

In [6]:
raise Exception()

Exception: 

In [7]:
cols = pd.Series(df.columns.format())

metrics = ["Temperature", "Precipitation in millimeters", "Distance (KM)"]  # 3 of these

ids = pd.concat([cols.loc[cols.str.match(".*Id.*")], cols.loc[cols.str.match(".*No.*")]]) # 3 of these

location = ["Pickup Lat", "Pickup Long", "Destination Lat", "Destination Long"] # 4 of these
times = cols.loc[cols.str.match(".*Time.*")] # 6(actually 4) of these

types = ["Vehicle Type", "Platform Type", "Personal or Business", ] # 3 of these


In [8]:
times

7                Placement - Time
8             Confirmation - Time
9        Arrival at Pickup - Time
10                  Pickup - Time
18    Time from Pickup to Arrival
dtype: object

In [9]:
ids

1      User Id
17    Rider Id
0     Order No
dtype: object

In [10]:
desc = pd.read_csv("variable_definitions.csv")
desc

,Variable Name,Description
0,Order No,Unique number identifying the order
1,User Id,Unique number identifying the customer on a pl...
2,Vehicle Type,"For this competition limited to bikes, however..."
3,Platform Type,"Platform used to place the order, there are 4 ..."
4,Personal or Business,Customer type
5,Placement - Day of Month,Placement - Day of Month i.e 1-31
6,Placement - Weekday (Mo = 1),Placement - Weekday (Monday = 1)
7,Placement - Time,Placement - Time - Time of day the order was p...
8,Confirmation - Day of Month,Confirmation - Day of Month i.e 1-31
9,Confirmation - Weekday (Mo = 1),Confirmation - Weekday (Monday = 1)
